## Cross validation odnosno unakrsna validacija

**Problem:** Mere kvaliteta modela zavise tj. variraju od podele na trening i test skup koja je nasumicna. <br>
**Cilj:** Da svaka od instanci dobije priliku da bude i u jednom i u drugom skupu. <br>
**Resenje:** Unakrsna validacija, kadgod to skupovi podataka i modeli dozvoljavaju, vise puta deli polazni skup podataka na skup za treniranje i testiranje.  <br> <br>
**k-Fold** je metod unakrsne validacije koji se ubedljivo najcesce korsiti: <img src='kfold.ppm'>

Na slici je prikazana k-Fold validacija sa 10 slojeva, drugim recima k je jednako 10. Imamo 10 krugova treniranja i evaluiranja naseg modela. U svakom *i*-tom krugu treniranja i evaluacije, sloj *i* se koristi za testiranje, a preostalih $k-1$ slojeva se koristi za treniranje modela. <br> Specijalan slucaj, kada je k jednako broju instanci u skupu, jeste `leave-one-out` metoda unakrsne validacije. Nju koristimo u slucaju malih skupova podataka kada nam je u cilju da iskoristimo sto vise instanci za treniranje.

Radi poredjenja koristimo iste podatke kao iz skripte za kNN algoritam komandom:

In [ ]:
%run kNN.ipynb

Klasu `KFold` koristimo za gore opisano particionisanje: 

In [ ]:
kfold = model_selection.KFold(n_splits=10) # broj slojeva preko parametra `n_splits`

In [ ]:
accuracy_scores = []

In [ ]:
# metodom `split` se generise se niz particija skupa koje sadrze indekse instanci
for train_indexes, test_indexes in kfold.split(X, y):
    # trening skup
    X_train = X.iloc[train_indexes]
    y_train = y[train_indexes]
    print(X_train.shape)
    # test skup
    X_test = X.iloc[test_indexes]
    y_test = y[test_indexes]
    print(X_test.shape)
    #priprema podataka za treniranje
    scaler = preprocessing.StandardScaler()
    scaler.fit(X_train)
    X_train_transformed = scaler.transform(X_train)
    X_test_transformed = scaler.transform(X_test)
    
    # treniranje
    model = neighbors.KNeighborsClassifier(n_neighbors=3)
    model.fit(X_train_transformed, y_train)
    
    # evaluacija
    score = model.score(X_test_transformed, y_test)
    
    accuracy_scores.append(score)

In [ ]:
accuracy_scores # evaluacija svakog sloja

Tacnost naseg modela racunamo kao prosek svih tacnosti dobijenih unakrsnom validacijom:

In [ ]:
accuracy = np.array(accuracy_scores).mean()

In [ ]:
print('Tacnost modela je: ', accuracy)

**Napomena:** Ne moze se svaka mera uproseciti!

### Sve na jednom mestu

Biblioteka `scikit-learn` nam daje `cross_val_score` koja vrsi unakrsnu validaciju. Ova funkcija od argumenata ocekuje model koji se evaluira, ceo skup podataka, funkciju za ocenu modela i broj slojeva, a za uzvrat vraca listu ocena modela na svakom sloju.

In [ ]:
accuracy_scores = model_selection.cross_val_score(neighbors.KNeighborsClassifier(n_neighbors=3), X, y, scoring='accuracy', cv=10)

In [ ]:
accuracy_scores

In [ ]:
accuracy_scores.mean()

**Paznja!!!** Dobili smo nizu tacnost modela.

**Zasto se to desava?** <br> To je posledica treniranja i testiranja modela na nestandardizovanim skupovima. 

Uvodimo novi pojam, pipeline, koji definise sve korake u procesu pripreme i treniranja modela.

In [ ]:
from sklearn import pipeline 

In [ ]:
knn_pipeline =  pipeline.make_pipeline(preprocessing.StandardScaler(), neighbors.KNeighborsClassifier(n_neighbors=3))

In [ ]:
# umesto modela pisemo pipeline
accuracy_scores_with_pipeline = model_selection.cross_val_score(knn_pipeline, X, y, scoring='accuracy', cv=10)

In [ ]:
accuracy_scores_with_pipeline

In [ ]:
accuracy_scores_with_pipeline.mean()

Tacnost i dalje blago odstupa. **Zasto?**
Nismo koristili bas identicno particionisanje skupa (pogledati dokumentaciju). Iako nije greska, da bismo reprodukovali, odnosno uporedili rezultate, mozemo parametru `cv` dati vrednost malocas napravljene klase `kfold`.

In [ ]:
accuracy_scores_with_cv = model_selection.cross_val_score(knn_pipeline, X, y, scoring='accuracy', cv=kfold)

In [ ]:
accuracy_scores_with_cv

In [ ]:
accuracy_scores_with_cv.mean()

### Finalni model

Finalni model je onaj model koji je dobijen treniranjem nad celim skupom podataka, a kao njegovu meru kvaliteta prijavljujemo gore izracunatu prosecnu tacnost i dalje ne evaluiramo.

In [ ]:
final_scaler = preprocessing.StandardScaler()
X = final_scaler.fit_transform(X) # obratite paznju na skup na kom se fituje skaliranje

In [ ]:
import pickle
with open('knn_scaler.scaler', 'wb') as pickle_file:
    pickle.dump(final_scaler, pickle_file)

In [ ]:
final_model = neighbors.KNeighborsClassifier(n_neighbors=3)

In [ ]:
final_model.fit(X, y)

In [ ]:
with open('knn_model.model', 'wb') as pickle_file:
    pickle.dump(final_model, pickle_file)